In [ ]:
from pathlib import Path
import sys
import pandas as pd
import matplotlib.pyplot as plt
sys.path.append(str(Path.cwd().parent))

import multiprocessing
import os
from loguru import logger
from src.utils.gcs_utils import set_environement_variable

is_env_variables_set = set_environement_variable()
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'path to auth.json file'

Using Default Env Variable:GCS
Using Default Env Variable:LOAD_MODELS
Using Default Env Variable:GCS_IMAGE_BUCKET
Using Default Env Variable:USER_DERMINATOR
Using Default Env Variable:PW_DERMINATOR


In [2]:
from src.utils.gcs_utils import *
bucket = get_bucket_gcs('oro-ds-test-bucket')

In [7]:
images=[]
labels=[]

In [8]:
def read_images_gcs(bucket, prefix='sdd_Debarpan/Images_danderm/', label = True):
    #set_gcs_env(gcs=1) #TODO

    iterator = bucket.list_blobs(delimiter="/", prefix=prefix)
    response = iterator._get_next_page_response()
    
    if 'items' in response.keys():
        num_of_images=len(response['items'])
        
        i=0
        while i<num_of_images:
            img_bucket_uri=response['items'][i]['name']

            images.append(img_bucket_uri)
            labels.append(label)
            i +=1

    elif 'prefixes' in response.keys():
        
        for c in response['prefixes']:
            if not c.endswith("/"):
                c += "/"

            read_images_gcs(bucket, prefix = c, label = label)
    else:
        print('ERROR')

In [ ]:
prefix = 'sdd_Debarpan/Images_danderm/moles/'
read_images_gcs(bucket, prefix = prefix, label = True)

In [ ]:
len(images)

In [11]:
prefix = 'sdd_Debarpan/Images_danderm/non_moles/'
read_images_gcs(bucket, prefix = prefix, label = False)

In [12]:
len(images)

2952

In [20]:
data = {'pathBucketImage' : images, 'labelledby' : ['KingsAndQueensOfDataScience']*len(images), 'reviewedby' : ['N/A']*len(images), 'mole' : labels, 'no_mole' : [not i for i in labels], 'dateInserted' : datetime.datetime.now().date(), 'ratios' : [0]*len(images)}
df = pd.DataFrame(data)

In [23]:
df.to_csv('dataset_1.0.csv')